# 機械学習スクラッチ　クラスタリング

# 【問題1】中心点の初期値を決める

入力されたデータから 
K
 個の中心点 
μ
1
 から 
μ
K
 の初期値を決めるコードを作成してください。
K
 は指定したクラスタ数です。


最もシンプルな初期値の決定方法は、データ点 
X
n
 の中からランダムに 
K
 個選ぶことです。今回はこれを実装してください。


K-meansの学習結果は中心点 
μ
 の初期値に影響を受けます。そのため、学習時には複数個の初期値で計算を行います。

In [5]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
from operator import itemgetter
np.random.seed(seed = 0)

In [6]:
from sklearn.datasets import make_blobs
X, _ = make_blobs(n_samples=100, n_features=2, centers=4, cluster_std=0.5, shuffle=True, random_state=0)

In [3]:
X

array([[ 0.72086751,  3.71347124],
       [-1.89468423,  7.96898545],
       [ 1.35678894,  4.36462484],
       [ 1.05374379,  4.49286859],
       [ 1.59141542,  4.90497725],
       [ 0.78260667,  4.15263595],
       [-1.95751686,  3.87291474],
       [-0.77354537,  7.87923564],
       [ 0.12313498,  5.27917503],
       [-1.43284669,  7.71577043],
       [-0.92819001,  7.02698199],
       [-1.74836345,  7.06307447],
       [-1.26789718,  7.25141327],
       [-0.98661744,  7.74968685],
       [-0.81984047,  7.50994722],
       [ 2.99684287,  0.22378413],
       [ 1.46870582,  1.86947425],
       [-0.33533163,  3.390122  ],
       [-1.86407034,  2.93379754],
       [ 2.62496786,  0.28025075],
       [ 2.11114739,  3.57660449],
       [-1.8219901 ,  7.61654999],
       [-1.91186205,  3.18750686],
       [ 2.28809874,  0.12954182],
       [ 0.5285368 ,  4.49723858],
       [-1.57613028,  2.58614312],
       [-0.565433  ,  3.65813966],
       [ 0.802314  ,  4.38196181],
       [ 2.79939362,

In [4]:
K = 3 #クラスタ数
X[np.random.choice(X.shape[0], K, replace=False), :]

array([[-0.565433  ,  3.65813966],
       [ 1.60161834,  0.92363636],
       [ 1.35678894,  4.36462484]])

In [5]:
u= X[np.random.choice(X.shape[0], K, replace=False), :]
u

array([[-1.86407034,  2.93379754],
       [ 2.64465731,  0.80770124],
       [ 1.42002502,  1.38236201]])

# 【問題2】SSEを求める関数の作成

クラスタ内誤差平方和（SSE, Sum of Squared Errors）を計算する関数を作成してください。


K-meansはこのSSEを最小化する 
r
n
k
 と 
μ
k
 を求めることが目的となります。複数個の初期値で計算したクラスタリング結果から、どれを最終的に採用するかを決める際にこのSSEを求める関数を使用します。



$
SSE = \sum_{n=1}^N \sum_{k=1}^K r_{nk} \|X_n - \mu_k\|^2
$

n
 : データ点のインデックス


k
 : クラスタのインデックス


X
n
 : 
n
番目のデータ点


μ
k
 : 
k
番目の中心点


r
n
k
 : データ点
X
n
がクラスタ
k
に所属していたら1、そうでなければ0

In [6]:
def SSE(k,X):
    X_size,n_features = X.shape #(100,2)
    
    # ランダムに重心の初期値を初期化
    centroids  = X[np.random.choice(X_size,k)]
    
    # 各データ所属クラスタ情報を保存する配列を用意
    cluster = np.zeros(X_size)
    
    # 入力データ全てに対して繰り返し
    for i in range(X_size):
        # データから各重心までの距離を計算（ルートを取らなくても大小関係は変わらないので省略）
        distances = np.sum((centroids - X[i]) ** 2, axis=1)# [a b k]
        # データの所属クラスタを距離の一番近い重心を持つものに更新
        cluster[i] = np.argsort(distances)[0] #[0 1 2 0 1 2 1 1 1 1 1 1 ・・・]
     
    
    # すべてのクラスタに対して距離を合計するSSE
    distances_sum = np.zeros(k)
    print(distances_sum)
    
    for j in range(k):
        distances_ = np.sum((centroids[j] - X[cluster == j]) ** 2, axis=1)
        distances_sum[j] = np.sum(distances_)
        print(np.sum((centroids[j] - X[cluster == j]) ** 2, axis=1))
        print(distances_sum)
        
    SSE = np.sum(distances_sum)
        
        
    return SSE
        

In [7]:
SSE(3,X)

[0. 0. 0.]
[ 0.1610424   0.51268696 17.44178915  5.50180627 16.18689489  0.54054793
 16.89745386  6.7068079  12.36477101 12.45802877  5.92474632  0.45397693
  7.26559394 10.7596904  10.97658155  8.96426593  7.74932442  9.78216779
  0.70137625  8.03787534  8.11567109  0.34832347  8.20316706  0.1712066
 13.57717459 12.14485233  0.        ]
[201.94782314   0.           0.        ]
[ 0.65128235 17.92502651  0.27586148  0.18330247 13.13332209  0.77577524
 14.20597247  8.52165475 11.76778855 10.81265328 12.87411215 10.89453462
  1.97197576 15.25508042  0.          1.90085692  0.08824269  8.58116019
 22.61960242 16.53519309  0.55496555  9.36152011 19.8503898   0.20735986
  1.40151435  0.07006069 12.01654644 15.99689666 13.62582319 10.84374678
 14.9439181  18.79721999  0.15705448 15.22040414  0.30386995 21.49765053
  0.21892874  0.23313533  0.18847463  0.82209923 16.74270886 13.70657319
 13.96591466  0.7046302 ]
[201.94782314 370.40480292   0.        ]
[2.64985046e+00 6.02752134e-01 1.00595354

664.4569134153629

# 【問題3】クラスタへの割り当て

全てのデータ点 
X
n
 を最も近い中心点 
μ
k
 に割り当てるコードを作成してください。


K-menasにおける 近い とは点と点のユークリッド距離が小さくなることです。ユークリッド距離とはピタゴラスの定理（三平方の定理）で求められるものですが、ベクトル 
p
,
q
 に対しては以下の数式で表現できます。

$
\|q-p\| = \sqrt{(q-p)\cdot(q-p)}
$

In [8]:
from numpy import linalg as LA

In [9]:
 c = np.array([[ 1, 2, 3],
               [-1, 1, 4]])
c

array([[ 1,  2,  3],
       [-1,  1,  4]])

In [10]:
 LA.norm(c, axis=0)

array([1.41421356, 2.23606798, 5.        ])

In [11]:
d = np.array([[ 1, 3, ],
               [-1, 4, ]])
LA.norm(d, axis=0)

array([1.41421356, 5.        ])

In [12]:
#####answer=問２でclusterに割り当てました#####

# 【問題4】中心点の移動

中心点 
μ
k
 を 
k
 番目のクラスタに割り当てられる全てのデータ点 
X
n
 の平均値（重心）に移動するコードを作成してください。


クラスタの割り当て 
r
n
k
 を固定した上でSSEを最小化していることになりますが、SSE自体を求める必要はありません。

In [13]:
def SSE_(k,X):
    X_size,n_features = X.shape #(100,2)
    
    # ランダムに重心の初期値を初期化
    centroids  = X[np.random.choice(X_size,k)]
    
    # 各データ所属クラスタ情報を保存する配列を用意
    cluster = np.zeros(X_size)
    """""
    追加したところ問4
    """""
    # 前の重心と比較するために、仮に新しい重心を入れておく配列を用意
    new_centroids = np.zeros((k, n_features))
    """""
     追加したところ問4
     """""
    
    # ループ上限回数まで繰り返し
    for epoch in range(max_iter):
        
    
        # 入力データ全てに対して繰り返し
        for i in range(X_size):
            # データから各重心までの距離を計算（ルートを取らなくても大小関係は変わらないので省略）
            distances = np.sum((centroids - X[i]) ** 2, axis=1)# [a b k]
            # データの所属クラスタを距離の一番近い重心を持つものに更新
            cluster[i] = np.argsort(distances)[0] #[0 1 2 0 1 2 1 1 1 1 1 1 ・・・]


        # すべてのクラスタに対して距離を合計するSSE
        distances_sum = np.zeros(k)

        for j in range(k):
            distances_ = np.sum((centroids[j] - X[cluster == j]) ** 2, axis=1)
            distances_sum[j] = np.sum(distances_)


        SSE = np.sum(distances_sum)
        """""
        追加したところ問4
        """""
        # すべてのクラスタに対して重心を再計算
        for j in range(k):#j=012
            new_centroids[j] = X[cluster==j].mean(axis=0)
        """""
        追加したところ問4
        """""
    print(new_centroids)
        
        
    return SSE
        

In [14]:
SSE_(3,X)

NameError: name 'max_iter' is not defined

# 【問題5】繰り返し

クラスタへの割り当てと中心点の移動を繰り返すことで学習を行うコードを作成してください。


次のどれかの条件を満たしたらこれを止めます。


繰り返しが指定した回数に到達する。（雛形では引数max_iter）
中心点 
μ
k
 と重心の差が指定した許容値以下になる。（雛形では引数tol

# 【問題6】異なる初期値で計算

問題5の繰り返しを、指定した回数初期値を変更して計算するコードを作成してください。回数は雛形では引数n_initで指定します。


複数計算した中でSSEが最小になったものが最終的な学習結果となるため、その時の中心点 
μ
k
 を保持しておきます。

In [30]:
class ScratchKMeans():
    """
    K-meansのスクラッチ実装

    Parameters
    ----------
    n_clusters : int
      クラスタ数
    n_init : int
      中心点の初期値を何回変えて計算するか
    max_iter : int
      1回の計算で最大何イテレーションするか
    tol : float
      イテレーションを終了する基準となる中心点と重心の許容誤差
    verbose : bool
      学習過程を出力する場合はTrue
    """
    def __init__(self, n_clusters, n_init, max_iter, tol, verbose=False):
        # ハイパーパラメータを属性として記録
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.tol = np.full(n_clusters, tol)
        self.true = None
        
        #tol_ = np.zeros(n_clusters)
        #print(self.tol)
        self.verbose = verbose
    def fit(self, X):
        """
        K-meansによるクラスタリングを計算
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習用データの特徴量
        """
    def _SSE(self,k,X):
        X_size,n_features = X.shape#(100,2)
        self.uk  = []
        self.sse = []
        #self.uk = np.zeros(self.n_init)
        print(self.uk)
        
        for roop in range(self.n_init):
            print(roop)
            print( "111")
        # ランダムに重心の初期値を初期化
 
            self.centroids  = X[np.random.choice(X_size,k)]

            # 各データ所属クラスタ情報を保存する配列を用意
            cluster = np.zeros(X_size)

            # 前の重心と比較するために、仮に新しい重心を入れておく配列を用意
            self.new_centroids = np.zeros((k, n_features))


            # ループ上限回数まで繰り返し
#             for epoch in range(self.max_iter):
            epoch = 0
            while epoch  < self.max_iter:
                print( "222")

                # 入力データ全てに対して繰り返し
                for i in range(X_size):
                    # データから各重心までの距離を計算（ルートを取らなくても大小関係は変わらないので省略）
                    distances = np.sum((self.centroids - X[i]) ** 2, axis=1)# [a b k]
                    # データの所属クラスタを距離の一番近い重心を持つものに更新
                    cluster[i] = np.argsort(distances)[0] #[0 1 2 0 1 2 1 1 1 1 1 1 ・・・]

                # すべてのクラスタに対して距離を合計するSSE
                distances_sum = np.zeros(k)
                
              
                #print(f"before_change{self.centroids}")
                # すべてのクラスタに対して重心を再計算
                
                for j in range(k):#j=012
                    self.new_centroids[j] = X[cluster==j].mean(axis=0)
                # 中心点 μ k と重心の差が指定した許容値以下になる
#                 print(f"centroid{self.centroids}")
#                 print(f"newcentroid{self.new_centroids}")
    #             print(f"diff{self.new_centroids - self.centroids}")
    
                #print((np.sum((self.new_centroids - self.centroids)**2, axis =1)))
                self.true = (np.sum((self.new_centroids - self.centroids)**2, axis =1) < self.tol)
                #print(self.true)
    #             print((np.sum((self.new_centroids - self.centroids)**2, axis =1) < self.tol))
    #             print(all(self.true))
                if all(self.true) or (epoch == self.max_iter-1 ):
                    print( "333")
#                     print("break")
                    self.uk.append(self.centroids)
                    epoch = self.max_iter

                    #self.uk[roop] = self.centroids.tolist()
                    print(self.uk)
                    for j in range(k):
                        distances_ = np.sum((self.centroids[j] - X[cluster == j]) ** 2, axis=1)
                        distances_sum[j] = np.sum(distances_)

                    SSE = np.sum(distances_sum)
                    self.sse.append(SSE)
                    print(self.sse)

                self.centroids =  self.new_centroids.copy() #参照渡しと値渡し
                epoch += 1
#                 print(f"undercentroid{self.new_centroids}")
                #print("-"*15)

                #if epoch == self.max_iter-1:
                    #self.uk.append(self.centroids)
#                     nt( "444")
#                     print(self.uk)
        
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        self.pre_centoids = itemgetter(np.argmin(self.sse))(self.uk)
        print(self.pre_centoids)
        
        return 

    def predict(self, X):
        """
        入力されたデータがどのクラスタに属するかを計算
        """
        #データ点ごとに予測したクラスタを格納するための箱
        self.pre_cluster = np.zeros(X.shape[0])
        # 入力データ全てに対して繰り返し
        for i in range(X.shape[0]):
            # データから各重心までの距離を計算（ルートを取らなくても大小関係は変わらないので省略）
            distances = np.sum((self.pre_centoids - X[i]) ** 2, axis=1)# [a b k]
            # データの所属クラスタを距離の一番近い重心を持つものに更新
            self.pre_cluster[i] = np.argsort(distances)[0] #[0 1 2 0 1 2 1 1 1 1 1 1 ・・・]
        
        
        return print(self.pre_cluster)
        
        
        

        return

In [31]:
skm = ScratchKMeans(3,n_init=5,max_iter=4,tol=0.1,verbose=False)
skm._SSE(3,X)

[]
0
111
222
222
333
[array([[ 1.99186903,  0.96561071],
       [-0.27678616,  3.72096344],
       [-1.26811733,  7.76378266]])]
[139.17097772033804]
1
111
222
222
333
[array([[ 1.99186903,  0.96561071],
       [-0.27678616,  3.72096344],
       [-1.26811733,  7.76378266]]), array([[ 1.99186903,  0.96561071],
       [-1.61318321,  3.06863899],
       [-0.18966435,  5.89872982]])]
[139.17097772033804, 250.63752944456343]
2
111
222
222
333
[array([[ 1.99186903,  0.96561071],
       [-0.27678616,  3.72096344],
       [-1.26811733,  7.76378266]]), array([[ 1.99186903,  0.96561071],
       [-1.61318321,  3.06863899],
       [-0.18966435,  5.89872982]]), array([[-1.61318321,  3.06863899],
       [-0.39550814,  6.28965918],
       [ 1.74370853,  1.62843289]])]
[139.17097772033804, 250.63752944456343, 245.53675483814908]
3
111
222
222
222
333
[array([[ 1.99186903,  0.96561071],
       [-0.27678616,  3.72096344],
       [-1.26811733,  7.76378266]]), array([[ 1.99186903,  0.96561071],
       [-1

# 【問題7】推定

新たなデータに対してどのクラスタに所属するかの推定の際は、学習時の1つ目のフェーズと同じことを行います。


データ点 
X
n
 と学習で決まった中心点 
μ
k
 に対し、 
S
S
E
 を最小化する 
r
n
k
 を選ぶ。

具体的には次のことです。


データ点 
X
n
 を最も近い 
μ
k
 に割り当てる。

In [32]:
skm.predict(X)

[1. 2. 1. 1. 1. 1. 1. 2. 1. 2. 2. 2. 2. 2. 2. 0. 0. 1. 1. 0. 1. 2. 1. 0.
 1. 1. 1. 1. 0. 0. 0. 2. 0. 0. 1. 1. 2. 0. 2. 1. 1. 1. 2. 1. 1. 2. 0. 1.
 1. 1. 1. 0. 1. 1. 1. 2. 2. 0. 0. 2. 2. 0. 0. 1. 0. 2. 1. 1. 0. 1. 2. 0.
 1. 2. 1. 1. 1. 1. 0. 0. 0. 2. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 2.
 2. 1. 2. 1.]


In [ ]:
def kmeans(k, X, max_iter=1):
    X_size,n_features = X.shape
    #print(X.shape)
    #print(X_size)
    #print(n_features)
    
    # ランダムに重心の初期値を初期化
    centroids  = X[np.random.choice(X_size,k)]
    print(centroids[0])
    #print(centroids)
    
    # 前の重心と比較するために、仮に新しい重心を入れておく配列を用意
    new_centroids = np.zeros((k, n_features))
    
    # 各データ所属クラスタ情報を保存する配列を用意
    cluster = np.zeros(X_size)
    
    # ループ上限回数まで繰り返し
    for epoch in range(max_iter):
        
        # 入力データ全てに対して繰り返し
        for i in range(X_size):
            
            # データから各重心までの距離を計算（ルートを取らなくても大小関係は変わらないので省略）
            distances = np.sum((centroids - X[i]) ** 2, axis=1)
          
            
            # データの所属クラスタを距離の一番近い重心を持つものに更新
            cluster[i] = np.argsort(distances)[0]
            #print(cluster[i])
            
        # すべてのクラスタに対して重心を再計算
        for j in range(k): #j=012
            new_centroids[j] = X[cluster==j].mean(axis=0)
            
            
        # もしも重心が変わっていなかったら終了
        if np.sum(new_centroids == centroids) == k:
            print("break")
            break
        centroids =  new_centroids
    return cluster

In [ ]:
cluster=kmeans(3, X )
cluster